In [ ]:
# GCP IAM에서 서비스 어카운트를 생성한 후 Create key 버튼을 통해 다운로드한 키 파일 경로가 필요합니다.
# GCP IAM에서 해당 서비스 어카운트에 AutoML admin 권한을 부여해야 합니다.
keyfile = '/Users/roy/blah/key.json'
%env GOOGLE_APPLICATION_CREDENTIALS $keyfile
! gcloud auth activate-service-account --key-file $keyfile

In [ ]:
# GCP 프로젝트 아이디를 기입합니다.
PROJECT_ID = ""

COMPUTE_REGION = "us-central1"

# AutoML 관련 정보
# AutoML 내부에서 생성할 데이터셋과 모델의 이름을 지정합니다.
AUTOML_DATASET = "house_price"
AUTOML_MODEL = "house_price_roy"
# AutoML Tables에서 모델을 학습시킬 시간 단위의 값으로 integer 형태의 값을 입력해야 합니다.
AUTOML_TRAIN_HOUR = 1

# Google Cloud Storage 관련 정보
# 학습시키거나 테스트할 데이터에 대한 CSV 파일 정보를 입력합니다.
GCS_BUCKET = "roy-automl"
GCS_DIR = "house_price"
GCS_TRAIN_DATA = "{}/train.csv".format(GCS_DIR)
GCS_TEST_DATA = "{}/test.csv".format(GCS_DIR)

# GCS에 업로드한 CSV 내용 가운데 지도학습 레이블이 될 컬럼을 지정합니다.
# 지도 학습 레이블 컬럼을 지정하면 해당 컬럼의 타입에 따라 어떤 머신러닝 모델을 써서 문제를 풀지 AutoML이 자동으로 결정합니다.
TARGET_COLUMN = "SalePrice"

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [ ]:
# !pip install google-cloud-automl

In [ ]:
from google.cloud import automl_v1beta1 as automl

In [ ]:
automl_client = automl.AutoMlClient()
tables_client = automl.TablesClient(project=PROJECT_ID, region=COMPUTE_REGION)

In [ ]:
dataset = tables_client.create_dataset(dataset_display_name=AUTOML_DATASET)

In [ ]:
GCS_TRAIN_DATA_URL = 'gs://{}/{}'.format(GCS_BUCKET, GCS_TRAIN_DATA)
GCS_TEST_DATA_URL = 'gs://{}/{}'.format(GCS_BUCKET, GCS_TEST_DATA)

In [ ]:
# 학습 데이터 로딩
response_importing_gcs_data = tables_client.import_data(
    dataset=dataset,
    gcs_input_uris=GCS_TRAIN_DATA_URL
)

In [ ]:
print('Importing GCS data to AutoML Tables.. ', response_importing_gcs_data.result())

In [ ]:
data_specified_notnull = tables_client.update_column_spec(
    dataset=dataset,
    column_spec_display_name=TARGET_COLUMN,
    nullable=False
)

In [ ]:
data_specified_label = tables_client.set_target_column(
    dataset=dataset,
    column_spec_display_name=TARGET_COLUMN,
)

In [ ]:
# 모델 학습
create_model_response = tables_client.create_model(
    AUTOML_MODEL,
    dataset=dataset,
    train_budget_milli_node_hours=AUTOML_TRAIN_HOUR*1000,
)

In [ ]:
model = create_model_response.result()
model_name = model.name

In [ ]:
# 학습한 모델로 예측
prediction_response = tables_client.batch_predict(
    model=model, 
    gcs_input_uris=GCS_TEST_DATA_URL,
    gcs_output_uri_prefix='gs://{}/{}'.format(GCS_BUCKET, GCS_DIR)
)

In [ ]:
print('Prediction result : ', prediction_response.result())